In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def simple_log_reg(X_train, y_train, X_test, y_test):
    
    clf = LogisticRegression().fit(X_train, y_train)

    train_score = clf.score(X_train, y_train)
    test_score = clf.score(X_test, y_test)

    train_pred = clf.predict_proba(X_train)
    test_pred = clf.predict_proba(X_test)
    
    return train_score,test_score,train_pred,test_pred

def plot_log_preds(df, X_train, X_test, train_preds, test_preds, x_col, y_col):
    plt.scatter(df[x_cols], df[y_col], label='True Vals')
    plt.scatter(X_train[x_cols], train_preds[:,1], label='Train Pred')
    plt.scatter(X_test[x_cols], test_preds[:,1], label='Test Pred')
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.title("Simple Logistic Regression Predictions")
    plt.legend()
    plt.show()

### Modeling on five states with most deaths (found through EDA): 
California, Florida, New York, Texas, Pensylvania

In [ ]:
df = pd.read_csv("medicare_aggregated_qid_2011_2012.csv")

states = ['CA', 'FL', 'NY', 'TX', 'PA']
state_df = df[df.statecode.isin(states)]

# sample random subsample
df_sub = state_df.sample(n=1000000)

df_sub = map_statecode(df_sub)

# remove columns not for modeling
df_sub2 = df_sub.drop(columns=['qid', 'pm25_ensemble', 'death'])

# split into train and test
X_train, X_test, y_train, y_test = train_test_split(df_sub2.drop(columns=['dead']), df_sub2.dead, test_size=0.3, random_state=42)

# standardize
non_binary_predictors = ['age','latitude','longitude', 'pm25_nn', 
       'ozone', 'poverty', 'popdensity', 'medianhousevalue', 'pct_blk',
       'medhouseholdincome', 'pct_owner_occ', 'hispanic', 'education',
       'smoke_rate', 'mean_bmi', 'tmmx', 'rmax', 'pr']

X_train_standardized, X_test_standardized = standardize(non_binary_predictors,X_train, X_test)

### Logistic Regression and Bootstrapping

In [ ]:
# first split into train and test set
train_df, test_df = train_test_split(state_df, test_size=0.2, random_state=42)

num_bootstraps = 300
num_sample = 1000000

model_coef = []
for i in range(num_bootstraps):
    # resample
    sub = train_df.sample(n=num_sample, replace=True)
    # map statecode
    sub2 = map_statecode(sub)
    # split into x and y
    x_boot_train = sub2.drop(columns=['dead'])
    y_boot_train = sub2.dead
    #standardize x
    x_boot_standardized = standardize(x_boot_train)
    # fit model
    model = LogisticRegression().fit(x_boot_standardized[['pm25_nn','ozone','tmmx','rmax']], y_boot_train)
    # save coefficients
    model_coef.append(model.coef_) 
    
model_coef = np.array(model_coef).reshape((num_bootstraps,4))

In [ ]:
def plot_betas(model_coef):
    fig, ax = plt.subplots(2,2, figsize = (18,10))
    ax = ax.ravel()
    for i in range(4):
        betavals = model_coef[:,i]
        betavals.sort()
        x1 = np.percentile(betavals,2.5)
        x2 = np.percentile(betavals,97.5)
        x = np.linspace(x1,x2,500)
        counts, bins = np.histogram(betavals)
        y = counts.max()
        ax[i].hist(model_coef[:,i],bins =10, color="#FF7E79",alpha=0.3,edgecolor='black', linewidth=1)
        ax[i].fill_between(x,y, color = '#007D66',alpha=0.2)
        # Prettify
        ax[i].set_ylim(0,27)
        ax[i].set_ylabel(f'Distribution of beta {i+1}',fontsize=18)
        ax[i].set_xlabel(f'Value of beta {i+1}',fontsize=18)
    #plt.xticks(fontsize=20)
    fig.suptitle(f'95 % confidence interval of coefficients', fontsize = 24)
    sns.despine()

plot_betas(model_coef)